In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Exploration of the dataset
We first start by checking the presence of missing values

In [2]:
dataset = pd.read_csv('wbc.csv').drop('id', axis=1)
print(f'Shape of the dataset : {dataset.shape}.\nPresence of NaNs:\n{dataset.isna().sum()}')

Shape of the dataset : (569, 31).
Presence of NaNs:
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


We now check the descriptive statistics of the dataset

In [9]:
dataset.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [3]:
X = dataset.drop('diagnosis', axis=1).values
y = dataset['diagnosis'].values
y = y=='M'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99, stratify=y)

# Feature Selection

In [4]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

ss = StandardScaler()
X_scale = ss.fit_transform(X)
selector = RFECV(LogisticRegression(max_iter=10000), step=1, cv=5)
selector.fit(X_scale, y)
print(selector.ranking_)
print(selector.support_)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


# Compare models performance

## Logistic regression

In [5]:
from sklearn.linear_model import LogisticRegression

steps = [('scale', StandardScaler()), ('logreg', LogisticRegression())]
params = {'logreg__penalty': ['l1', 'l2'],
         'logreg__C': np.linspace(0.1, 10.0, 30),
         'logreg__solver': ['saga'],
         'logreg__max_iter': [10000]}
pipeline = Pipeline(steps)
grid_logreg = GridSearchCV(pipeline, param_grid=params, cv=5)

grid_logreg.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('logreg', LogisticRegression())]),
             param_grid={'logreg__C': array([ 0.1       ,  0.44137931,  0.78275862,  1.12413793,  1.46551724,
        1.80689655,  2.14827586,  2.48965517,  2.83103448,  3.17241379,
        3.5137931 ,  3.85517241,  4.19655172,  4.53793103,  4.87931034,
        5.22068966,  5.56206897,  5.90344828,  6.24482759,  6.5862069 ,
        6.92758621,  7.26896552,  7.61034483,  7.95172414,  8.29310345,
        8.63448276,  8.97586207,  9.31724138,  9.65862069, 10.        ]),
                         'logreg__max_iter': [10000],
                         'logreg__penalty': ['l1', 'l2'],
                         'logreg__solver': ['saga']})

## Random Forest Classifier

In [33]:
from sklearn.ensemble import RandomForestClassifier

params = {
    'n_estimators': [500],
    'max_depth': [6],
    'max_features': [1]
        #'n_estimators': np.arange(100, 600, 100),
        #'max_depth': np.arange(2,16,2),
        #'max_features': [1.0, 'sqrt', 'log2',1],
    }

grid_rfc = GridSearchCV(RandomForestClassifier(random_state=99), param_grid = params, cv=5)

grid_rfc.fit(X_train, y_train)
print(grid_rfc.score(X_test, y_test), grid_rfc.best_params_, grid_rfc.best_score_)

0.956140350877193 {'max_depth': 6, 'max_features': 1, 'n_estimators': 500} 0.9472527472527472


## Support Vector Machine

In [28]:
from sklearn.svm import SVC

steps = [('scale', StandardScaler()), ('svc', SVC(random_state=99))]
params = {
        'svc__C': np.linspace(0.01, 1.0, 30),
        'svc__kernel': ['linear'],
        'svc__degree': np.arange(2,10,1)
}
pipeline = Pipeline(steps)

grid_svc = GridSearchCV(pipeline, param_grid=params, cv=5)

grid_svc.fit(X_train, y_train)

print(grid_svc.score(X_test, y_test), grid_svc.best_params_)

0.9649122807017544 {'svc__C': 0.04413793103448276, 'svc__degree': 2, 'svc__kernel': 'linear'}


## K Neighbors Classifier

In [27]:
from sklearn.neighbors import KNeighborsClassifier

steps = [('scale', StandardScaler()), ('knn', KNeighborsClassifier())]
params = {
        'knn__n_neighbors': np.arange(1,15),
        'knn__weights': ['uniform', 'distance']
}
pipeline=Pipeline(steps)

grid_knn = GridSearchCV(pipeline, param_grid=params, cv=5)

grid_knn.fit(X_train, y_train)

print(grid_knn.best_score_, grid_knn.best_params_, grid_knn.score(X_test, y_test))

0.9670329670329672 {'knn__n_neighbors': 9, 'knn__weights': 'uniform'} 0.956140350877193


## Naive Bayes

In [48]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)

print(gnb.score(X_test, y_test))

0.9210526315789473


## Ensemble Learning

In [47]:
from sklearn.metrics import accuracy_score

y_pred = (np.where(grid_rfc.predict(X_test),1,0) + np.where(grid_svc.predict(X_test),1,0) + np.where(grid_knn.predict(X_test),1,0))//2
print(accuracy_score(y_test, y_pred))

0.9649122807017544
